# Tabular Playground Series - Feb 2022

This dataset is aimed to practive ML skills by using the genomic analysis technique data with some data compression and data loss.

ATATGGCCTT will be represented as A2T4G2C2 instead of the sequence.

Let's start with checking the data set and let's find out some direction for analysis.

### Aim:
Read the notebook from competition expert and learn the way to deal with data, model training

To perform well in the training and know more about the data set, the paper for this competition was suggested to read through.

https://www.frontiersin.org/articles/10.3389/fmicb.2020.00257/full

The model used in the paper was Passive-Aggressive Classifier (PA, linear machine learning algorithm); Extra Trees Classifier (ET, decision tree algorithm); Gaussian Naïve Bayes (GNB, Naïve Bayes algorithm); Linear Discriminant Analysis (LDA, discriminant analysis algorithm); and the neural network (NN, whose default is 100 layers and the number of nodes determined by the number of input features).

Here is some hints that we can work on.

In [1]:
# libraries set up
import pandas as pd
import numpy as np
import sklearn
import seaborn as sns
import matplotlib.pyplot as plt

For the preparation work, it can be compressed into few lines of code instead of few sections

In [2]:
# read the data, drop the duplicate and pop the target for train set
train_path = 'data/train.csv'
test_path = 'data/test.csv'
# read the data, drop the duplicate at the same time
x_train = pd.read_csv(train_path, index_col = 0).drop_duplicates()
x_test = pd.read_csv(test_path, index_col = 0)
y_train = x_train.pop('target').to_frame()

# one hot code for the target
y_train_ohe = pd.get_dummies(y_train['target'])
y_toname = {x:y for x,y in enumerate(y_train_ohe.columns)}
y_toint = {y:x for x,y in enumerate(y_train_ohe.columns)}
y_train['target_num'] = y_train['target'].map(y_toint)

KNN
- As the notebook said the the performance is better without cross-validatoin. It preferred to use the full training set once & it can be observed from my previous notebook since there is no obvious change after we have implemented the cross-validation. (Experimental noise & mutations was found in the data set from the paper. The expiermental noise was appeared in both train & test set but the mutation was found in the test set)

- Experimental noise (Sampling noise) was tackled with large sample size
- Mutation (Sampling bias) was the main issue but it can/ cannot be avoided into the testing set.

In [4]:
from sklearn.neighbors import KNeighborsClassifier
# build the K-neighbors classifier model
model = KNeighborsClassifier(n_jobs = -1, metric = 'manhattan', weights = 'distance', n_neighbors = 2)
# fit the data into the model
model.fit(x_train, y_train['target_num'])
# get the probability for the prediction to check with the distribution
y_test_prob = pd.DataFrame(model.predict_proba(x_test), index = x_test.index, columns = y_train_ohe.columns)

To get a great/ good testing set, the probabilities in all the class should be balanced. Most notebooks did this by adding different constants to each class.

In [27]:
# check class distribution
# return the column index for each sample
y_test_knn = y_test_prob.apply(lambda x: x.argmax(), axis = 1)
# turn the index into the name
y_test_knn = y_test_knn.map(y_toname).rename('target').reset_index()
# return the count with relative frequency
display(y_test_knn.target.value_counts(normalize=True))

Escherichia_fergusonii      0.10444
Campylobacter_jejuni        0.10197
Streptococcus_pneumoniae    0.10049
Klebsiella_pneumoniae       0.10048
Streptococcus_pyogenes      0.10045
Salmonella_enterica         0.10022
Bacteroides_fragilis        0.10003
Enterococcus_hirae          0.09964
Staphylococcus_aureus       0.09855
Escherichia_coli            0.09373
Name: target, dtype: float64

The probabilities is not that balanced especially Escherichia_coli.

This part is quite confuse since the editor try to use the quantile to add on more probabilties for 10% of observation. The idea is not sure but it did a great job to balance the probabilties distribution. It work well to deal with the bias.

In [35]:
display(y_test_prob.quantile(0.90))

Bacteroides_fragilis        0.500000
Campylobacter_jejuni        0.613518
Enterococcus_hirae          0.500000
Escherichia_coli            0.375595
Escherichia_fergusonii      0.534749
Klebsiella_pneumoniae       0.505897
Salmonella_enterica         0.500000
Staphylococcus_aureus       0.404429
Streptococcus_pneumoniae    0.518519
Streptococcus_pyogenes      0.509804
Name: 0.9, dtype: float64

In [46]:
# divide all the probabilties with 90% quantile
y_test_prob /= y_test_prob.quantile(0.9)

In [48]:
# get the 50% probability -> result the 10% of observation having more that 50% probability for each class
y_test_prob /= 2

In [50]:
y_test_prob.quantile(0.9)

Bacteroides_fragilis        0.5
Campylobacter_jejuni        0.5
Enterococcus_hirae          0.5
Escherichia_coli            0.5
Escherichia_fergusonii      0.5
Klebsiella_pneumoniae       0.5
Salmonella_enterica         0.5
Staphylococcus_aureus       0.5
Streptococcus_pneumoniae    0.5
Streptococcus_pyogenes      0.5
Name: 0.9, dtype: float64

In [52]:
y_test_knn = y_test_prob.apply(lambda x : x.argmax(), axis = 1)
y_test_knn = y_test_knn.map(y_toname).rename('target').to_frame()
y_test_knn[['target']].to_csv('submission_knn.csv')
display(y_test_knn.target.value_counts(normalize = True))

Campylobacter_jejuni        0.10082
Streptococcus_pyogenes      0.10053
Streptococcus_pneumoniae    0.10024
Salmonella_enterica         0.10016
Bacteroides_fragilis        0.10004
Escherichia_fergusonii      0.09986
Staphylococcus_aureus       0.09984
Enterococcus_hirae          0.09967
Escherichia_coli            0.09948
Klebsiella_pneumoniae       0.09936
Name: target, dtype: float64

After using the KNN model for the classification, the accuracy (0.98644)is very high compared to my previous model.

By adjusting the result with quantile, the accuracy (0.98810) is much better.

#### Thing needed to be investigated for this part:

1) Why not cross-validation?

2) manhattan metric & inverse-distance weights and only 2 neighbors

3) Why manhattan appears to be more robust against outlier mutations than the euclidean distance (L2).

4) Why use the quantile to rescale?

#### Another thing needed to be investigated before next part:
    
1) Extra tree classifier (Many notebook used it and the editor as work with it for better accuracy as well)

2) GCD (Another technique used by different notebook as well)

3) PCA to visualize the training set & predicted test set. (Accuracy visualization)

Reference video:

1) https://www.youtube.com/watch?v=1VgevfcOmyE (Extra tree classifier)

2) https://www.youtube.com/watch?v=yHwneN6zJmU (Euclidean Algorithm)